# Ensemble Algorithm

In [37]:
import numpy as np
import tensorflow as tf
import keras
from keras import applications, Model
from keras.models import load_model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.metrics import make_scorer
from keras.utils.np_utils import to_categorical
import random
import sklearn
import pandas as pd
import warnings

warnings.simplefilter('ignore')

# Import Data

In [38]:
train_data_dir = 'Data/Training_Data'
validation_data_dir = 'Data/Validation_Data'
test_data_dir= 'Data/Test_Data'

train_labels_file = 'Labels/training_labels.npy'
validation_labels_file = 'Labels/validation_labels.npy'
test_labels_file = 'Labels/test_labels.npy'

img_width, img_height = 224, 224
NB_CLASSES = 11

In [39]:
train_labels = np.load(open(train_labels_file, 'rb'))
validation_labels = np.load(open(validation_labels_file, 'rb'))
test_labels = np.load(open(test_labels_file, 'rb'))

In [40]:
print('Training Data : ' + str(len(train_labels)) + ' Images')
print('Validation Data : ' + str(len(validation_labels)) + ' Images')
print('Test Data : ' + str(len(test_labels)) + ' Images')

Training Data : 307 Images
Validation Data : 74 Images
Test Data : 18 Images


## Ensemble the models

In [41]:
models = list()

In [42]:
vgg_model = load_model('Models/model_vgg16.h5')
models.append(vgg_model)

In [43]:
inception_model = load_model('Models/model_inception.h5')
models.append(inception_model)

In [44]:
resnet_model = load_model('Models/model_Resnet.h5')
models.append(resnet_model)

In [45]:
densenet_model = load_model('Models/model_Densenet.h5')
models.append(densenet_model)

## Results

In [46]:
datagen = ImageDataGenerator()

In [47]:
test_generator = datagen.flow_from_directory(
            test_data_dir,
            target_size=(img_width, img_height),
            batch_size=len(test_labels),
            class_mode=None,
            shuffle=False)

Found 18 images belonging to 11 classes.


In [49]:
# make predictions
test_predictions = [model.predict_generator(test_generator, 1, verbose=1) for model in models]
# calculate average
test_predictions = np.mean(test_predictions, axis=0)

1/1 [==============================] - 2s 2s/step


In [50]:
test_predictions = np.asarray(list(map(str,np.argmax(test_predictions,axis=1)))).reshape(-1,1)

In [51]:
test_labels = np.asarray(test_labels).reshape(-1,1)

In [52]:
test_acc, test_acc_op = tf.metrics.accuracy(test_labels, test_predictions)

In [53]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [54]:
test_accuracy = sess.run(test_acc_op)

In [55]:
print('\nTest accuracy : ' + str(test_accuracy))


Test accuracy : 1.0
